### Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import openai
import json
from datetime import datetime

### Convert from raw data to xlsx
The code snippet provided is a Python function named flatten_json that takes a single argument `json_obj`, which is expected to be a dictionary (potentially representing a JSON object). The function's purpose is to "flatten" the input JSON object so that nested dictionaries are transformed into a single-level dictionary with keys representing the original nested paths. \
In summary, this function transforms a potentially nested JSON object into a flat dictionary (dataframe, excel format), where each key represents a path through the original nested structure, making it easier to work with in contexts where nested data structures are inconvenient or unsupported.

In [ ]:
# Chuẩn bị hàm để làm phẳng dữ liệu JSON:

def flatten_json(json_obj):
    flattened_data = {}
    for key, value in json_obj.items():
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                new_key = f'{key}_{sub_key}'
                flattened_data[new_key] = sub_value
        else:
            flattened_data[key] = value
    return flattened_data


Each month in the year data will be looping through to get the data file of each day, the data will be converted into dataframe using `flatten_json` function and recorded back into day to day data in the same manner.

In [ ]:
year_folder = '../dataRaw/108/2022'
output_year_folder = '../output/108/2022'
all_data = []

# Duyệt qua từng thư mục con trong thư mục cha (năm)
for month in range(1, 13):
    month_folder = os.path.join(year_folder, f'{month:02d}').replace('\\', '/')
    output_month_folder = os.path.join(output_year_folder, f'{month:02d}').replace('\\', '/')

    # Tạo thư mục cho tháng trong output nếu chưa tồn tại
    os.makedirs(output_month_folder, exist_ok=True)

    # Duyệt qua từng file JSON trong thư mục của từng tháng
    for file_name in os.listdir(month_folder):
        if file_name.endswith('.json'):
            file_path = os.path.join(month_folder, file_name)

            # Đọc file JSON
            with open(file_path, 'r') as file:
                json_data = json.load(file)

            flattened_json_data = flatten_json(json_data)

            rows = []
            for key, value in flattened_json_data.items():
                if isinstance(value, dict):
                    for timestamp, timestamp_value in value.items():
                        row = next((row for row in rows if row.get('timestamp') == timestamp), None)
                        if row is None:
                            row = {'timestamp': timestamp}
                            rows.append(row)
                        row[key] = timestamp_value
                else:
                    for row in rows:
                        row[key] = value

            df = pd.DataFrame(rows)
            print(df[:20])

            output_file_path = os.path.join(output_month_folder, file_name.replace('.json', '.xlsx'))
            df.to_csv(output_file_path, index=False)



### Set the operating hours

In [ ]:
# Define operating hours based on the information provided
operating_hours = {
    "Monday": ["08:00", "18:00"],
    "Tuesday": ["08:00", "18:00"],
    "Wednesday": ["08:00", "18:00"],
    "Thursday": ["08:00", "18:00"],
    "Friday": ["08:00", "18:00"],
    "Saturday": ["08:00", "18:00"],
    "Sunday": ["08:00", "18:00"],
    # Assuming Public Holiday information will be used accordingly when needed
    "Public Holiday": ["08:00", "12:00"]
}

hvac_data_types = ['room_temp', 'dew_temp', 'slab_temp', 'fan_stat']

setpoint_values = {
    'room_temp': {'value': 22, 'range': 2},
    'dew_temp': {'value': 10, 'range': 2},
    'slab_temp': {'value': 20, 'range': 2},
    # 'fan_stat' does not have a setpoint
}

### Function to check if a given datetime is within operating hours
The `is_within_operating_hours` function checks if a given datetime, found in a `row` dictionary under the key `'datetime'`, falls within specified operating hours. The `operating_hours` parameter is a dictionary where each key is a day of the week, and its value is a list containing two strings: the start and end times for the operating hours on that day. The function first converts the `row['datetime']` to a day name to check if it exists within `operating_hours`. If so, it then checks whether the `row['datetime']` is between the start and end times specified for that day. If the datetime falls within the operating hours, the function returns `True`; otherwise, it returns `False`.


In [ ]:
def is_within_operating_hours(row, operating_hours):
    day_of_week = row['datetime'].strftime('%A')
    if day_of_week in operating_hours:
        start_time = pd.to_datetime(operating_hours[day_of_week][0])
        end_time = pd.to_datetime(operating_hours[day_of_week][1])
        return start_time.time() <= row['datetime'].time() <= end_time.time()
    return False

### Functions to calculate statistics
The provided code snippet defines two functions related to statistical analysis of a dataset, specifically designed to handle numeric data and accommodate scenarios involving non-numeric values:

1. `calculate_statistics(values, sp_value=None, sp_range=None)`: This function computes basic statistics (average, minimum, maximum) for a list of values, optionally evaluating how many of these values fall outside a specified range around a setpoint value (`sp_value`) if both `sp_value` and `sp_range` are provided. It handles non-numeric values gracefully by converting all input values to floats, silently ignoring any values that cannot be converted.

2. `calculate_ideal_temperature_percentage(temp_values, sp_value, sp_range)`: This auxiliary function calculates the percentage of values within a specified range (`sp_range`) around a setpoint value (`sp_value`), effectively determining the proportion of values that are considered within an "ideal" range. It is used by `calculate_statistics` to compute the `outside_sp` statistic, which represents the percentage of time the temperature (or other measured values) is outside the ideal range.

Both functions leverage pandas for data manipulation, ensuring that the operations can handle large datasets efficiently and flexibly.

In [ ]:
# Function to calculate statistics, handling non-numeric values
def calculate_statistics(values, sp_value=None, sp_range=None):
    # Ensure the input is a pandas Series
    values_series = pd.Series(values)
    # Convert all values to floats, ignoring non-convertible values
    valid_values = pd.to_numeric(values_series, errors='coerce').dropna()
    stats = {'average': None, 'min': None, 'max': None, 'outside_sp': None}
    if len(valid_values) > 0:
        stats['average'] = valid_values.mean()
        stats['min'] = valid_values.min()
        stats['max'] = valid_values.max()
        if sp_value is not None and sp_range is not None:
            stats['outside_sp'] = 100 - calculate_ideal_temperature_percentage(valid_values, sp_value, sp_range)
    return stats

# New function to calculate the percentage of time the temperature is within the ideal range
def calculate_ideal_temperature_percentage(temp_values, sp_value, sp_range):
    ideal_min = sp_value - sp_range
    ideal_max = sp_value + sp_range
    within_ideal = temp_values.apply(lambda x: ideal_min <= x <= ideal_max)
    return (within_ideal.sum() / len(within_ideal)) * 100

In [ ]:
def calculate_fan_on_percentage(values):
    fan_on_values = pd.Series(values)
    return (fan_on_values.sum() / len(fan_on_values)) * 100

### Function to process each file, including conversion of temperature columns to numeric types
The `process_file` function is designed to analyze HVAC (Heating, Ventilation, and Air Conditioning) data from an Excel file, given a path to the file (`file_path`) and a dictionary of operating hours (`operating_hours`). The function performs several key operations:

1. **Load Data**: Utilizes pandas to load data from an Excel file. If the loaded data is empty, the function returns `-1`.

2. **Datetime Conversion**: Converts timestamps in the data to datetime objects for easier manipulation and comparison.

3. **Operating Hours Classification**: Applies the `is_within_operating_hours` function to each row to determine if the recorded data falls within the specified operating hours. This classification results in a new boolean column (`is_operating_hour`) in the dataframe.

4. **Data Segmentation**: Splits the dataset into two separate dataframes based on whether the data falls within operating hours or not.

5. **Statistics Calculation**: Initializes a dictionary to store computed statistics for both operating and non-operating hours. It then processes various HVAC data types (e.g., temperature, fan status) present in the dataset:

    - For each HVAC data type, it identifies relevant columns and converts their values to numeric types, handling non-numeric values gracefully.
    - Specifically for fan status (`fan_stat`), it calculates the percentage of time the fan is on during both operating and non-operating hours.
    - For temperature-related data types, it calculates basic statistics (average, minimum, maximum) and the percentage of values outside a specified range around a setpoint value (`outside_sp`). This involves computing how often the temperature falls outside an "ideal" range defined for operating and non-operating hours.

6. **Return Value**: The function returns a dictionary containing the calculated statistics for both operating and non-operating hours across all relevant HVAC data types.

In [ ]:
def process_file(file_path, operating_hours):
    # Load data
    data = pd.read_excel(file_path)

    if data.empty:
        return -1

    # Convert timestamp to datetime
    data['datetime'] = pd.to_datetime(data['timestamp'], unit='s')

    # Determine if each row is within operating hours
    data['is_operating_hour'] = data.apply(lambda row: is_within_operating_hours(row, operating_hours), axis=1)

    # Split the data into operating and non-operating hours
    data_operating_hours = data[data['is_operating_hour']]
    data_non_operating_hours = data[~data['is_operating_hour']]

    # Initialize dictionary to store statistics for operating and non-operating hours
    stats = {
        'operating': {},
        'non_operating': {}
    }

    # Process all relevant HVAC data types
    for data_type in hvac_data_types:
        data_cols = [col for col in data.columns if data_type in col]
        if not data_cols:  # Skip if no columns found for this data type
            continue

        # Convert columns to numeric types
        for col in data_cols:
            data[col] = pd.to_numeric(data[col], errors='coerce')

        if data_type == 'fan_stat':
            # Calculate fan on percentage for operating hours
            stats['operating'][data_type] = {'on_percentage': calculate_fan_on_percentage(data_operating_hours[data_cols].stack())}
            # Calculate fan on percentage for non-operating hours
            stats['non_operating'][data_type] = {'on_percentage': calculate_fan_on_percentage(data_non_operating_hours[data_cols].stack())}
        else:
            # Calculate statistics for operating hours for temperature data types
            sp_value = setpoint_values[data_type]['value']
            sp_range = setpoint_values[data_type]['range']
            # Calculate statistics including outside_sp for temperature data types
            stats['operating'][data_type] = calculate_statistics(data_operating_hours[data_cols].stack(), sp_value, sp_range)
            stats['non_operating'][data_type] = calculate_statistics(data_non_operating_hours[data_cols].stack(), sp_value, sp_range)
    # print(stats)
    return stats

### Function to format the output statistics
The `format_statistics_output` function processes and formats statistics for various HVAC data types, excluding fan statistics (`fan_stat`). It takes a dictionary `overall_stats` containing pre-calculated statistics for different HVAC data types as input. The function performs the following operations:

1. **Initialization**: Initializes an empty dictionary `formatted_stats` to store the formatted statistics.

2. **Data Processing**: Iterates through each HVAC data type defined in `hvac_data_types`. For each type (excluding `fan_stat`):
   - Converts `None` values to `numpy.nan` for safe numerical processing. This conversion is done for averages, minimums, maximums, and outside setpoint percentages (`outside_sp`), ensuring that `numpy` functions can be applied without issues.
   - Computes and stores the mean average, minimum, maximum, and mean outside setpoint percentage for each data type. If no valid data points are available (i.e., all values are `None` or the list is empty), it assigns `-1` as a placeholder.

3. **Fan Statistics Special Handling**: For `fan_stat`, it similarly converts `None` to `numpy.nan` but focuses on calculating the overall on percentage.

4. **Return Value**: Returns `formatted_stats`, a dictionary containing the formatted and computed statistics for each HVAC data type. This dictionary provides a standardized way to report statistics, handling missing or invalid data gracefully and ensuring consistency across different data types.


In [ ]:
def format_statistics_output(overall_stats):
    formatted_stats = {}
    for data_type in hvac_data_types:
        if data_type != 'fan_stat':
            # Convert None to np.nan for safe processing with numpy functions
            averages = [x if x is not None else np.nan for x in overall_stats[data_type]['average']]
            mins = [x if x is not None else np.nan for x in overall_stats[data_type]['min']]
            maxes = [x if x is not None else np.nan for x in overall_stats[data_type]['max']]
            outside_sps = [x if x is not None else np.nan for x in overall_stats[data_type]['outside_sp']]

            formatted_stats[data_type] = {
                'average': np.nanmean(averages) if averages else -1,
                'min': np.nanmin(mins) if mins else -1,
                'max': np.nanmax(maxes) if maxes else -1,
                'outside_sp': np.nanmean(outside_sps) if outside_sps else -1
            }
        else:
            # For fan_stat, convert None to np.nan and calculate the overall on percentage
            on_percentages = [x if x is not None else np.nan for x in overall_stats[data_type]['average']]
            formatted_stats[data_type] = {
                'on_percentage': np.nanmean(on_percentages) if on_percentages else -1
            }
    return formatted_stats

### Function to format statistics into e2e nlg "source" structure
The `stats_to_e2e_source` function formats HVAC statistics into a string suitable for end-to-end (e2e) source data representation, incorporating both operational type (e.g., operating or non-operating hours) and optional summary information. The function takes three parameters:

- `stats`: A dictionary where each key represents a data type (e.g., `fan_stat`, temperature statistics) and each value is another dictionary containing specific statistics for that data type (average, minimum, maximum, outside setpoint percentage).
- `operating_type`: A string indicating the type of operation (e.g., "Operating", "Non-Operating") the statistics apply to.
- `summary`: An optional string containing a summary or additional information to be included at the end of the formatted string.

The function processes each data type within `stats` as follows:

1. **Fan Statistics Handling**: For `fan_stat`, it specifically formats the on percentage, ensuring numerical values are displayed with two decimal places, or "N/A" if the data is unavailable.

2. **Temperature Statistics**: For other data types (assumed to be temperature-related), it formats average, minimum, maximum values, and the percentage of readings outside the specified setpoint range. Each statistic is formatted to ensure numerical values are properly displayed (or "N/A" if unavailable), and concatenates them with descriptive labels.

3. **String Construction**: Constructs the final string by combining the operational type label with the formatted statistics for each data type. If a `summary` is provided, it is appended at the end of the string.

4. **Return Value**: Returns the formatted string, which represents a comprehensive summary of the HVAC statistics for a specific operational type, optionally including a summary or additional information. This function provides a structured and readable way to present complex statistical data in a single string format, facilitating easy interpretation and communication of key data points.


In [ ]:
def stats_to_e2e_source(stats, operating_type, summary):
    parts = []
    for data_type, data_stats in stats.items():
        if data_type == 'fan_stat':  # Handle fan_stat differently
            on_percentage = data_stats.get('on_percentage')
            part = f"{data_type} on_percentage : {on_percentage:.2f}%" if on_percentage is not None else f"{data_type} on_percentage : N/A"
        else:  # For temperature stats
            average = data_stats.get('average')
            min_val = data_stats.get('min')
            max_val = data_stats.get('max')
            outside_sp = data_stats.get('outside_sp')

            # Prepare strings, handling None values
            average_str = f"{average:.2f}" if average is not None else " N/A"
            min_str = str(min_val) if min_val is not None else " N/A"
            max_str = str(max_val) if max_val is not None else " N/A"
            outside_sp_str = f"{outside_sp:.2f}%" if outside_sp is not None else " N/A"

            part = (f"{data_type} average : {average_str} | "
                    f"{data_type} min : {min_str} | "
                    f"{data_type} max : {max_str} | "
                    f"{data_type} outside_sp : {outside_sp_str}")
        parts.append(part)

    if summary == None:
        source_str = f"name : {operating_type} Hours Statistics | " + " | ".join(parts)
    else:
        source_str = f"name : {operating_type} Hours Statistics | " + " | ".join(parts) + '", "target": "' + summary + '"}'

    return source_str

### Generating a summary based on the provided template and example
The `generate_summary` function creates a comprehensive summary of HVAC data for a specific date extracted from a given file path. This function performs several key steps:

1. **Date Extraction**: It extracts the date from the `file_path` using a regular expression that matches a specific date format (`YYYY-MM-DD`) within the file name. If a date is found, it formats this date into a more readable format (`%B %d, %Y`), otherwise, it defaults to using "date" as a placeholder.

2. **Summary Generation**: Utilizing the extracted date and provided data for both operating and non-operating hours (`data` and `data_non` dictionaries, respectively), the function constructs a detailed summary.

3. **Error Handling**: If the function encounters a `TypeError` during summary generation (potentially due to missing or invalid data), it prints an error message and the problematic data, returning a generic error summary instead.

4. **Return Value**: The function returns the generated summary as a string. This summary provides a detailed overview of the HVAC system's performance on a specific date, differentiating between operating and non-operating hours for a thorough analysis.

This function first create a fixed template of reporting before put forward to the next step using LLMs.


In [ ]:
def generate_summary(data, data_non, file_path):
    # Extract date from file_path
    date_match = re.search(r'data_(\d{4}-\d{2}-\d{2})\.xlsx', file_path)
    if date_match:
        report_date = datetime.strptime(date_match.group(1), '%Y-%m-%d').strftime('%B %d, %Y')
    else:
        report_date = "date"

    try:
        summary = (f"On {report_date}, during operating hours, room temperatures fluctuated from a minimum of {data['room_temp']['min']:.2f}°C to a maximum of {data['room_temp']['max']:.2f}°C, "
                   f"averaging {data['room_temp']['average']:.2f}°C. The temperature was outside the set point range {data['room_temp']['outside_sp']:.2f}% of the time. "
                   f"Dew point temperatures during operating hours ranged from {data['dew_temp']['min']:.2f}°C to {data['dew_temp']['max']:.2f}°C, averaging {data['dew_temp']['average']:.2f}°C. "
                   f"Slab temperatures varied between {data['slab_temp']['min']:.2f}°C and {data['slab_temp']['max']:.2f}°C during operating hours, with an average of {data['slab_temp']['average']:.2f}°C. "
                   f"The fan operated at an percent of {data['fan_stat']['on_percentage']:.2f}% of the time during operating hours. "
                   f"Outside of operating hours, minimum and maximum temperatures were {data_non['room_temp']['min']:.2f}°C and "
                   f"{data_non['room_temp']['max']:.2f}°C respectively, with an average temperature of {data_non['room_temp']['average']:.2f}°C. "
                   f"The average dew point temperature was {data_non['dew_temp']['average']:.2f}°C, while slab temperatures averaged {data_non['slab_temp']['average']:.2f}°C. "
                   f"The fan's operation outside operating hours at {data_non['fan_stat']['on_percentage']:.2f}%."
                   )
    except TypeError as e:
        print(f"Error processing data: {e}")
        print(f"Data causing error: {data}")
        summary = "Error generating summary for this entry."
    return summary

### API OpenAI call for creating more templates and wording variations
The `call_API` function rephrases HVAC data summaries using the OpenAI API, ensuring data accuracy while varying the wording. Key steps include:

- **API Key Configuration**: Sets your OpenAI API key (user-supplied) in `openai.api_key` for authentication.
- **Instructions Preparation**: Forms instructions for the API to rephrase the `summary` for more templates basedon the fixed version, emphasizing accurate data representation and converting specific percentages into words.
- **API Request**: Calls the OpenAI API (`gpt-3.5-turbo-instruct`) with the instructions, applying creativity control (`temperature=0.7`) and a token limit.
- **Output Formatting**: Ensures the rephrased summary is on a single line by replacing newline characters with spaces.
- **Return**: Outputs the single-line, rephrased summary.

This function allows for automated rephrasing of summaries, suitable for varied presentation or reporting needs, requiring an OpenAI API key for operation.


In [ ]:
def call_API(summary):
    # Set your OpenAI API key
    openai.api_key = ""

    instructions = (
        "Rephrase the paragraph with wording variations. Ensure all data remains accurate and unchanged from the input paragraph. "
        # "For example, if the input paragraph is: During operating hours, room temperatures fluctuated from a minimum "
        # "of 20.50°C to a maximum of 23.50°C, averaging 21.76°C. The temperature was outside the set point "
        # "range 0.00% of the time. Dew point temperatures during operating hours ranged from 9.90°C to 13.20°C, "
        # "averaging 11.94°C. Slab temperatures varied between 21.70°C and 25.40°C during operating hours, with "
        # "an average of 23.42°C. The fan operated at an percent of 82.72% of the time during operating hours. "
        # "Outside of operating hours, minimum and maximum temperatures were 21.00°C and 27.50°C respectively, "
        # "with an average temperature of 23.62°C. The average dew point temperature was 13.11°C, while slab "
        # "temperatures averaged 23.73°C. The fan's operation outside operating hours at 21.54%, output: "
        # "Throughout the operating hours, the range of room temperatures observed was from a low of 20.50°C to a high "
        # "of 23.50°C, with the mean temperature being 21.76°C. There were no instances where the temperature deviated "
        # "from the preset range. The dew point temperature varied from 9.90°C to 13.20°C, with an average of 11.94°C. "
        # "The temperature of the slab fluctuated between 21.70°C and 25.40°C, averaging at 23.42°C. The fan was operational "
        # "82.72% of the time during these hours. Outside the operating hours, the temperature span was from 21.00°C to 27.50°C, "
        # "with a mean of 23.62°C. The dew point temperature on average was 13.11°C, and the average slab temperature was "
        # "recorded at 23.73°C. Fan usage was reduced to 21.54% outside of operating hours. If the input paragraph is: During operating hours, "
        # "temperatures fluctuated from a minimum of 22.00°C to a maximum of 28.20°C, averaging 25.42°C. The temperature "
        # "was outside the set point range 67.90% of the time. Dew point temperatures during operating hours ranged "
        # "from 12.20°C to 17.10°C, averaging 14.76°C. Slab temperatures varied between 21.20°C and 25.20°C during "
        # "operating hours, with an average of 23.15°C. The fan operated at an percent of 13.33% of the time during "
        # "operating hours. Outside of operating hours, minimum and maximum temperatures were 21.50°C and 28.00°C "
        # "respectively, with an average temperature of 24.75°C. The average dew point temperature was 14.28°C, while "
        # "slab temperatures averaged 23.12°C. The fan's operation outside operating hours at 13.33%, output: "
        # "During business hours, the temperature in the room varied from a low of 22.00°C to a high of 28.20°C, "
        # "with an average of 25.42°C. This temperature was beyond the desired range 67.90% of the time. Dew point "
        # "temperatures ranged between 12.20°C and 17.10°C, averaging at 14.76°C. The temperature of the slab showed "
        # "variations from 21.20°C to 25.20°C, averaging at 23.15°C. The fan was active for 13.33% of the operating "
        # "hours. After hours, room temperatures spanned from 21.50°C to 28.00°C, averaging at 24.75°C. The mean dew "
        # "point temperature was 14.28°C, with slab temperatures averaging at 23.12°C. The fan's activity remained the "
        # "same at 13.33% outside operating hours."
        "Any 0.00% or 100% should be replaced and represented its meaning by words, not numbers 0.00% or 100%, and all the other "
        "numbers should be kept unchanged." + summary
    )

    completion = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",
        prompt=instructions,
        temperature=0.7,
        max_tokens=250
    )

    # Ensure the output is on a single line by replacing newlines if necessary
    single_line_output = completion.choices[0].text.replace("\n", " ")

    print(single_line_output)
    return single_line_output


### Main function for data processing
The provided script automates the processing and aggregation of HVAC system statistics from Excel files (.xlsx) organized by month and year. Here's a breakdown of its key components and workflow:

1. **Initialization**: Two dictionaries (`all_operating_stats` and `all_non_operating_stats`) are created to store aggregated statistics (average, minimum, maximum, and outside setpoint percentage) for different HVAC data types across all files.

2. **Directory Setup**: Variables `year_folder` and `output_year_folder` define the input and output directory paths for a specific year.

3. **Monthly Processing Loop**:
   - For each month (1-12), input and output directories are defined.
   - The script ensures the output directory exists (creating it if necessary).
   - It then iterates through all Excel files in the input directory, processing each file to extract HVAC statistics.

4. **File Processing**:
   - The `process_file` function analyzes each file to generate operating and non-operating hours statistics.
   - These statistics are aggregated across all processed files, with separate handling for `fan_stat` data.

5. **Statistics Aggregation**: For each data type, statistics are appended to the respective lists in the initialized dictionaries. All the zones are aggreagated so if you want to make changes for specific purpose, you might have to change the code.

6. **Statistics Formatting**:
   - The `format_statistics_output` function formats the aggregated statistics for both operating and non-operating hours into a more readable form.
   - Summaries of these statistics are generated using the `generate_summary` function, which is designed to create a comprehensive report for each file.

7. **Summary Generation and Output**:
   - Summaries and detailed statistics are formatted for end-to-end (e2e) natural language generation (NLG) purposes.
   - The results are printed to the console and saved to a text file in the corresponding output directory, replacing the original Excel file extension with `.txt`.

This script streamlines the analysis and reporting of HVAC data, facilitating efficient monthly and yearly reviews of system performance through automated data processing, aggregation, and summary generation.

**!!! MAKE SURE YOU UNCOMMENT THE call_API SO THAT THE API IS CALLED TO GENERATE MORE TEMPLATES VARIATIONS**


In [ ]:
# Initialize the structure to store aggregated statistics
all_operating_stats = {data_type: {'average': [], 'min': [], 'max': [], 'outside_sp': []} for data_type in hvac_data_types}
all_non_operating_stats = {data_type: {'average': [], 'min': [], 'max': [], 'outside_sp': []} for data_type in hvac_data_types}


year_folder = '../output/108/2022'
output_year_folder = '../train_stsm_1/108/2022'

# ... (other function definitions like is_within_operating_hours, calculate_statistics, etc.)

# Modify this part to read from directories
for month in range(1, 13):
    month_folder = os.path.join(year_folder, f'{month:02d}')
    output_month_folder = os.path.join(output_year_folder, f'{month:02d}')

    # Create output month folder if it doesn't exist
    os.makedirs(output_month_folder, exist_ok=True)

    # Process all .xlsx files within the month folder
    for file_name in os.listdir(month_folder):
        file_path = ''
        if file_name.endswith('.xlsx'):
            file_path = os.path.join(month_folder, file_name)
            stats = process_file(file_path, operating_hours)

            if stats == -1:
                continue

            # Statistic for a month
            for hvac_data_type, data_stats in stats['operating'].items():
                if hvac_data_type != 'fan_stat':
                    # Append temperature-related statistics and outside_sp
                    all_operating_stats[hvac_data_type]['average'].append(data_stats['average'])
                    all_operating_stats[hvac_data_type]['min'].append(data_stats['min'])
                    all_operating_stats[hvac_data_type]['max'].append(data_stats['max'])
                    all_operating_stats[hvac_data_type]['outside_sp'].append(data_stats['outside_sp'])
                else:
                    # For fan_stat, append the on percentage directly
                    all_operating_stats[hvac_data_type]['average'].append(data_stats['on_percentage'])

            for hvac_data_type, data_stats in stats['non_operating'].items():
                if hvac_data_type != 'fan_stat':
                    # Append temperature-related statistics and outside_sp
                    all_non_operating_stats[hvac_data_type]['average'].append(data_stats['average'])
                    all_non_operating_stats[hvac_data_type]['min'].append(data_stats['min'])
                    all_non_operating_stats[hvac_data_type]['max'].append(data_stats['max'])
                    all_non_operating_stats[hvac_data_type]['outside_sp'].append(data_stats['outside_sp'])
                else:
                    # For fan_stat, append the on percentage directly
                    all_non_operating_stats[hvac_data_type]['average'].append(data_stats['on_percentage'])

        # Format the output for clear understanding
        formatted_operating_stats = format_statistics_output(all_operating_stats)
        formatted_non_operating_stats = format_statistics_output(all_non_operating_stats)

        summary = generate_summary(formatted_operating_stats, formatted_non_operating_stats, file_path)
        print("\n", summary)

        # UNCOMMENT THIS LINE OF CODE TO GENERATE MORE TEMPLATES IN ORDER TO TRAIN THE MODEL WITH MORE VARIATIONS.
        # summary = call_API(summary)

        # Convert the statistics to e2e nlg "source" format
        e2e_source_operating = stats_to_e2e_source(formatted_operating_stats, "Operating", None)
        e2e_source_non_operating = stats_to_e2e_source(formatted_non_operating_stats, "Non-Operating", summary)

        result = '{"source": "'+ e2e_source_operating + " | " + e2e_source_non_operating
        print(result)


        output_file_path = os.path.join(output_month_folder, file_name.replace('.xlsx', '.txt'))

        os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

        # Ghi nội dung vào tệp tin
        with open(output_file_path, 'w') as file:
            file.write(result)


 On January 01, 2022, during operating hours, room temperatures fluctuated from a minimum of 22.00°C to a maximum of 28.20°C, averaging 25.42°C. The temperature was outside the set point range 67.90% of the time. Dew point temperatures during operating hours ranged from 12.20°C to 17.10°C, averaging 14.76°C. Slab temperatures varied between 21.20°C and 25.20°C during operating hours, with an average of 23.15°C. The fan operated at an percent of 13.33% of the time during operating hours. Outside of operating hours, minimum and maximum temperatures were 21.50°C and 28.00°C respectively, with an average temperature of 24.75°C. The average dew point temperature was 14.28°C, while slab temperatures averaged 23.12°C. The fan's operation outside operating hours at 13.33%.
{"source": "name : Operating Hours Statistics | room_temp average : 25.42 | room_temp min : 22.0 | room_temp max : 28.2 | room_temp outside_sp : 67.90% | dew_temp average : 14.76 | dew_temp min : 12.2 | dew_temp max : 17.1 


 On January 02, 2022, during operating hours, room temperatures fluctuated from a minimum of 22.00°C to a maximum of 28.50°C, averaging 25.47°C. The temperature was outside the set point range 71.23% of the time. Dew point temperatures during operating hours ranged from 12.20°C to 17.10°C, averaging 14.77°C. Slab temperatures varied between 21.20°C and 26.30°C during operating hours, with an average of 23.68°C. The fan operated at an percent of 13.33% of the time during operating hours. Outside of operating hours, minimum and maximum temperatures were 21.50°C and 28.50°C respectively, with an average temperature of 25.08°C. The average dew point temperature was 14.55°C, while slab temperatures averaged 23.67°C. The fan's operation outside operating hours at 13.33%.
{"source": "name : Operating Hours Statistics | room_temp average : 25.47 | room_temp min : 22.0 | room_temp max : 28.5 | room_temp outside_sp : 71.23% | dew_temp average : 14.77 | dew_temp min : 12.2 | dew_temp max : 17.1 

C:\Users\phamh\AppData\Local\Temp\ipykernel_2180\3577419115.py:3: RuntimeWarning: invalid value encountered in longlong_scalars
  return (fan_on_values.sum() / len(fan_on_values)) * 100



 On January 24, 2022, during operating hours, room temperatures fluctuated from a minimum of 20.50°C to a maximum of 29.50°C, averaging 23.06°C. The temperature was outside the set point range 18.96% of the time. Dew point temperatures during operating hours ranged from 9.90°C to 19.40°C, averaging 14.73°C. Slab temperatures varied between 20.90°C and 26.70°C during operating hours, with an average of 22.91°C. The fan operated at an percent of 47.78% of the time during operating hours. Outside of operating hours, minimum and maximum temperatures were 20.00°C and 29.50°C respectively, with an average temperature of 23.83°C. The average dew point temperature was 14.78°C, while slab temperatures averaged 23.08°C. The fan's operation outside operating hours at 15.34%.
{"source": "name : Operating Hours Statistics | room_temp average : 23.06 | room_temp min : 20.5 | room_temp max : 29.5 | room_temp outside_sp : 18.96% | dew_temp average : 14.73 | dew_temp min : 9.9 | dew_temp max : 19.4 | 

### Function to merge files into one file only to build train dataset

In [ ]:
def merge_files(folder_path, output_file):
    # Lấy danh sách các file trong folder và sắp xếp
    files = sorted([file for file in os.listdir(folder_path) if file.endswith('.txt')])

    print(files)

    # Mở file output để ghi dữ liệu
    with open(output_file, 'w') as outfile:
        # Duyệt qua từng file và ghi nội dung vào file output
        for filename in files:
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as infile:
                data = infile.read()
                if data == None:
                    continue
                outfile.write(data)
                outfile.write('\n') # Thêm dòng mới giữa các file

### Merge until get one file

In [ ]:
# MERGE DAY TO MONTH

folder_path = f'../train_stsm_1/108/2022'
output_folder = f'../train_stsm_1/new_summary/108/2022/'

os.makedirs(os.path.dirname(output_folder), exist_ok=True)

for month in range (1, 13):
    file_path = f'../train_stsm_1/108/2022/{month:02d}'
    output_file = f'../train_stsm_1/new_summary/108/2022/{month:02d}.txt'

    merge_files(file_path, output_file)


['data_2022-01-01.txt', 'data_2022-01-02.txt', 'data_2022-01-03.txt', 'data_2022-01-04.txt', 'data_2022-01-05.txt', 'data_2022-01-06.txt', 'data_2022-01-07.txt', 'data_2022-01-08.txt', 'data_2022-01-09.txt', 'data_2022-01-10.txt', 'data_2022-01-11.txt', 'data_2022-01-12.txt', 'data_2022-01-14.txt', 'data_2022-01-15.txt', 'data_2022-01-16.txt', 'data_2022-01-17.txt', 'data_2022-01-18.txt', 'data_2022-01-19.txt', 'data_2022-01-20.txt', 'data_2022-01-21.txt', 'data_2022-01-22.txt', 'data_2022-01-23.txt', 'data_2022-01-24.txt', 'data_2022-01-25.txt', 'data_2022-01-26.txt', 'data_2022-01-27.txt', 'data_2022-01-28.txt', 'data_2022-01-29.txt', 'data_2022-01-30.txt', 'data_2022-01-31.txt']


['data_2022-02-01.txt', 'data_2022-02-02.txt', 'data_2022-02-03.txt', 'data_2022-02-04.txt', 'data_2022-02-05.txt', 'data_2022-02-06.txt', 'data_2022-02-07.txt', 'data_2022-02-08.txt', 'data_2022-02-09.txt', 'data_2022-02-10.txt', 'data_2022-02-11.txt', 'data_2022-02-12.txt', 'data_2022-02-13.txt', 'data_2022-02-14.txt', 'data_2022-02-15.txt', 'data_2022-02-16.txt', 'data_2022-02-17.txt', 'data_2022-02-18.txt', 'data_2022-02-19.txt', 'data_2022-02-20.txt', 'data_2022-02-21.txt', 'data_2022-02-22.txt', 'data_2022-02-23.txt', 'data_2022-02-24.txt', 'data_2022-02-25.txt', 'data_2022-02-26.txt', 'data_2022-02-27.txt', 'data_2022-02-28.txt']
['data_2022-03-01.txt', 'data_2022-03-02.txt', 'data_2022-03-03.txt', 'data_2022-03-04.txt', 'data_2022-03-05.txt', 'data_2022-03-06.txt', 'data_2022-03-07.txt', 'data_2022-03-08.txt', 'data_2022-03-09.txt', 'data_2022-03-10.txt', 'data_2022-03-11.txt', 'data_2022-03-12.txt', 'data_2022-03-13.txt', 'data_2022-03-14.txt', 'data_2022-03-15.txt', 'data_202

In [ ]:
# MERGE MONTH TO YEAR
folder_path = f'../train_stsm_1/new_summary/108/2022'
output_file = f'../train_stsm_1/new_summary/merge/108_2022.json'

os.makedirs(os.path.dirname(output_file), exist_ok=True)

merge_files(folder_path, output_file)


['01.txt', '02.txt', '03.txt', '04.txt', '05.txt', '06.txt', '07.txt', '08.txt', '09.txt', '10.txt', '11.txt', '12.txt']


In [ ]:
# # MERGE YEAR

# folder_path = f'../train_stsm_1/new_summary/merge/'
# output_file = f'../train_stsm_1/new_summary/train/2022.json'

# os.makedirs(os.path.dirname(output_file), exist_ok=True)

# merge_files(folder_path, output_file)


### Split data into train, test, validation chunks

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

def split_csv_file(file_path, train_ratio, test_ratio, valid_ratio):
    # Đọc file CSV
    data = pd.read_json(file_path, lines=True)

    # Tính toán tỉ lệ cho test và valid
    test_valid_ratio = test_ratio + valid_ratio
    test_ratio_adjusted = test_ratio / test_valid_ratio
    print(test_valid_ratio, test_ratio_adjusted)
    # Chia dữ liệu thành train và phần còn lại
    train_data, test_valid_data = train_test_split(data, test_size=test_valid_ratio, random_state=42)

    # Chia phần còn lại thành test và valid
    test_data, valid_data = train_test_split(test_valid_data, test_size=test_ratio_adjusted, random_state=42)

    # Lưu dữ liệu vào các file mới
    train_data.to_json('../train_stsm_1/new_summary/train/train.json', orient='records', lines=True)
    test_data.to_json('../train_stsm_1/new_summary/train/test.json', orient='records', lines=True)
    valid_data.to_json('../train_stsm_1/new_summary/train/valid.json', orient='records', lines=True)

# Sử dụng hàm
file_path = '../train_stsm_1/new_summary/train/108_2022.json'
split_csv_file(file_path, 0.6, 0.2, 0.2)


0.4 0.5
